<a href="https://colab.research.google.com/github/davidparede/i2a2autonometa/blob/main/AutonoMeta_projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutonoMeta - Ferramenta de análise automática de documentos fiscais

Trabalho realizado para o curso de IA do I2A2 - grupo AutonoMeta, com o objetivo de permitir a análise de dados fiscais de forma fácil e otimizada, atráves do uso de linguagem natural. Todo projeto foi criado usando tecnologia open source e usando parametros de clean code. Os códigos foram desenvolvidos em python, usou-se autogen como framework e o colab como IDE.

#Etapa 1: Instalação de Dependências e Importações

In [73]:
pip install autogen google-generativeai

# Etapa 2: Importações e Configuração Inicial

In [74]:
import os
import zipfile
import requests
import pandas as pd
from autogen import AssistantAgent, UserProxyAgent
from IPython.display import display, Markdown
import ipywidgets as widgets
from google.colab import userdata
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, Markdown
import traceback
import pandas as pd

#CONFIGURAÇÃO ---

É uma boa prática de desenvolvimento definir constantes no início para fácil manutenção, seguindo parametros de microsserviço.

In [75]:
# Configurar variáveis de ambiente
ENDPOINT_API = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
MODEL_NAME = "gemini-2.0-flash"

# SEGURANÇA
Para fins de segurança, a chave de API foi criptografada

In [76]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

#Etapa 3: Preparação dos Dados e importação de documentos fiscais

In [77]:
 import pandas as pd
import os

# Definição do diretório onde os arquivos CSV estão localizados

# Nomes dos arquivos CSV
ARQUIVO_CABECALHO = '/content/202401_NFs_Cabecalho.csv'
ARQUIVO_ITENS = '/content/202401_NFs_Itens.csv'

def carregar_dados(diretorio, arquivo):
    """
    Carrega um arquivo CSV em um DataFrame pandas.

    Args:
        diretorio (str): O diretório onde o arquivo está localizado.
        arquivo (str): O nome do arquivo CSV.

    Returns:
        pd.DataFrame: O DataFrame carregado.
    """
    caminho_completo = os.path.join(diretorio, arquivo)
    try:
        df = pd.read_csv(caminho_completo)
        return df
    except FileNotFoundError:
        print(f"Erro: O arquivo {arquivo} não foi encontrado no diretório {diretorio}.")
        return None


# Etapa 4: Configuração do Agente AutoGen

In [78]:
CONTEXTO_SISTEMA = f"""
Você é um analista de dados especialista em notas fiscais, preciso e direto.
Sua tarefa é responder às perguntas do usuário usando os DataFrames do Pandas fornecidos.
Os DataFrames disponíveis são `cabecalho_df` e `itens_df`.

Schema do `cabecalho_df`: {list(cabecalho_df.columns) if cabecalho_df is not None else "DataFrame not loaded"}
Schema do `itens_df`: {list(itens_df.columns) if itens_df is not None else "DataFrame not loaded"}

Para responder, você deve:
1.  Analisar a pergunta do usuário.
2.  Gerar e executar o código Python necessário para encontrar a resposta.
3.  Apresentar a resposta final em linguagem natural, de forma clara e objetiva.
4.  Não mostre o código Python na sua resposta final, apenas o resultado.
"""

# Configuração do LLM
llm_config = {
    "config_list": [{
        "model": MODEL_NAME,
        "base_url": ENDPOINT_API,
        "api_key": GOOGLE_API_KEY,
    }],
    "cache_seed": None # Desativa o cache para garantir novas respostas
}

# Set the environment variable for AutoGen's OpenAI client
os.environ["OPENAI_API_KEY"] = GOOGLE_API_KEY

In [79]:
# Criação do agente assistente com o contexto aprimorado
assistant = AssistantAgent(
    name="Analista_Fiscal",
    system_message=CONTEXTO_SISTEMA,
    llm_config=llm_config
)

# Criação do agente proxy do usuário
user_proxy = UserProxyAgent(
    name="Usuario_Proxy",
    human_input_mode="NEVER", # O agente nunca pedirá intervenção humana
    max_consecutive_auto_reply=2, # Limita o número de respostas automáticas para evitar loops
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").rstrip(),
    code_execution_config={"work_dir": "analise", "use_docker": False}
)

In [80]:


CONTEXTO_SISTEMA = f"""
Você é um analista de dados especialista em notas fiscais, preciso e direto.
Sua tarefa é responder às perguntas do usuário usando os DataFrames do Pandas fornecidos.
Os DataFrames disponíveis são `cabecalho_df` e `itens_df`.

Schema do `cabecalho_df`: {list(cabecalho_df.columns) if cabecalho_df is not None else "DataFrame not loaded"}
Schema do `itens_df`: {list(itens_df.columns) if itens_df is not None else "DataFrame not loaded"}

Para responder, você deve:
1.  Analisar a pergunta do usuário.
2.  Gerar e executar o código Python necessário para encontrar a resposta.
3.  Apresentar a resposta final em linguagem natural, de forma clara e objetiva.
4.  Não mostre o código Python na sua resposta final, apenas o resultado.
"""

# Configuração do LLM (sem alterações, mas agora usa a chave de forma segura)
llm_config = {
    "config_list": [{
        "model": 'gemini-2.0-flash',
        "base_url": ENDPOINT_API,
        "api_key": os.environ.get("GOOGLE_API_KEY"), # Use OPENAI_API_KEY here
    }],
    "cache_seed": None # Desativa o cache para garantir novas respostas
}

# Set the environment variable for AutoGen's OpenAI client
os.environ["OPENAI_API_KEY"] = GOOGLE_API_KEY


# Criação do agente assistente com o contexto aprimorado
assistant = AssistantAgent(
    name="Analista_Fiscal",
    system_message=CONTEXTO_SISTEMA,
    llm_config=llm_config
)

# Criação do agente proxy do usuário
user_proxy = UserProxyAgent(
    name="Usuario_Proxy",
    human_input_mode="NEVER", # O agente nunca pedirá intervenção humana
    max_consecutive_auto_reply=2, # Limita o número de respostas automáticas para evitar loops
    is_termination_msg=lambda x: "TERMINATE" in x.get("content", "").rstrip(),
    code_execution_config={"work_dir": "analise", "use_docker": False}
)

# Etapa 5: Interface do Usuário Interativa

In [ ]:
# Widgets para interface
pergunta_input = widgets.Text(placeholder="Digite sua pergunta...")
botao_enviar = widgets.Button(description="Enviar")
saida = widgets.Output()

def on_button_click(b):
    with saida:
        saida.clear_output()
        pergunta = pergunta_input.value
        print(f"Processando: {pergunta}")

        # Iniciar chat com formatação controlada
        user_proxy.initiate_chat(
            assistant,
            message=f"""
            Dados disponíveis:
            - Cabeçalho (cabecalho_df): {list('/content/202401_NFs_Cabecalho.csv'),
            - Itens (itens_df): {list('/content/202401_NFs_Itens.csv'),
            Pergunta do usuário: {pergunta}

            Instruções:
            1. Analise os dataframes relevantes
            2. Execute e retorne apenas
               - Resposta principal
               - Método utilizado
               - Valores numéricos destacados
            3. Focando apenas na resposta em linguagem natural
            """
        )

        # Capturar e formatar última resposta
        ultima_resposta = assistant.last_message()['content']

        # Filtragem de conteúdo
        partes_relevantes = [
            linha for linha in ultima_resposta.split('\n')
            if not linha.startswith('```')
        ]

        # Exibir formatado
        display(Markdown(f"""
        **📝 Resposta:**
        {''.join(partes_relevantes[1:-1]) if len(partes_relevantes) > 2 else ''.join(partes_relevantes)}

        _ℹ️ Dados processados em {pd.Timestamp.now().strftime('%d/%m %H:%M')}_
        """))
# Associa a função ao evento de clique do botão
botao_enviar.on_click(on_button_click)

# Exibe a interface final
print("Sou AutonoMeta, como posso lhe ajudar hoje?.")
display(widgets.VBox([pergunta_input, botao_enviar, saida]))